In [1]:
%pip install scikit-learn numpy pandas matplotlib seaborn basemap phik sweetviz kaggle setuptools

Note: you may need to restart the kernel to use updated packages.


In [16]:
# import warnings
# warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

In [19]:
df = pd.read_csv('spotify-most-streamed-songs/Spotify Most Streamed Songs.csv')
df.head(5)

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,cover_url
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,B,Major,80,89,83,31,0,8,4,Not Found
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,C#,Major,71,61,74,7,0,10,4,https://i.scdn.co/image/ab67616d0000b2730656d5...
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,F,Major,51,32,53,17,0,31,6,https://i.scdn.co/image/ab67616d0000b273e85259...
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,A,Major,55,58,72,11,0,11,15,https://i.scdn.co/image/ab67616d0000b273e787cf...
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,A,Minor,65,23,80,14,63,11,6,https://i.scdn.co/image/ab67616d0000b273ab5c9c...


# Clear up 

In [20]:
df['streams'] = pd.to_numeric(df['streams'].str.replace(',', ''), errors='coerce')
print(f'Всего нечисловых значений: {df.streams.isna().sum()}')

Всего нечисловых значений: 1


In [21]:
df = df.dropna(subset=['streams'])
print(f'Всего нечисловых значений: {df.streams.isna().sum()}')

Всего нечисловых значений: 0


In [22]:
df['in_deezer_playlists'] = pd.to_numeric(df['in_deezer_playlists'].str.replace(',', '.'), errors='coerce')

In [23]:
df['in_shazam_charts'] = pd.to_numeric(df['in_shazam_charts'].str.replace(',', '.'), errors='coerce')
df = df.dropna(subset=['in_shazam_charts'])

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 902 entries, 0 to 952
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   track_name            902 non-null    object 
 1   artist(s)_name        902 non-null    object 
 2   artist_count          902 non-null    int64  
 3   released_year         902 non-null    int64  
 4   released_month        902 non-null    int64  
 5   released_day          902 non-null    int64  
 6   in_spotify_playlists  902 non-null    int64  
 7   in_spotify_charts     902 non-null    int64  
 8   streams               902 non-null    float64
 9   in_apple_playlists    902 non-null    int64  
 10  in_apple_charts       902 non-null    int64  
 11  in_deezer_playlists   902 non-null    float64
 12  in_deezer_charts      902 non-null    int64  
 13  in_shazam_charts      902 non-null    float64
 14  bpm                   902 non-null    int64  
 15  key                   816 no

In [ ]:
df['artist(s)_name'] = df['artist(s)_name'].str.split(
    r'& |, ').explode('artist(s)_name')

In [26]:
cat_cols = df.select_dtypes(include='object')
cat_cols.head(5)

,track_name,artist(s)_name,key,mode,cover_url
0,Seven (feat. Latto) (Explicit Ver.),Latto,B,Major,Not Found
1,LALA,Jung Kook,C#,Major,https://i.scdn.co/image/ab67616d0000b2730656d5...
2,vampire,Myke Towers,F,Major,https://i.scdn.co/image/ab67616d0000b273e85259...
3,Cruel Summer,Olivia Rodrigo,A,Major,https://i.scdn.co/image/ab67616d0000b273e787cf...
4,WHERE SHE GOES,Taylor Swift,A,Minor,https://i.scdn.co/image/ab67616d0000b273ab5c9c...


In [27]:
cat_cols.shape

(902, 5)

In [28]:
cat_cols.describe()

,track_name,artist(s)_name,key,mode,cover_url
count,902,902,816,902,902
unique,892,476,11,2,501
top,Let It Snow! Let It Snow! Let It Snow!,The Weeknd,C#,Major,Not Found
freq,2,31,115,521,219


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 902 entries, 0 to 952
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   track_name            902 non-null    object 
 1   artist(s)_name        902 non-null    object 
 2   artist_count          902 non-null    int64  
 3   released_year         902 non-null    int64  
 4   released_month        902 non-null    int64  
 5   released_day          902 non-null    int64  
 6   in_spotify_playlists  902 non-null    int64  
 7   in_spotify_charts     902 non-null    int64  
 8   streams               902 non-null    float64
 9   in_apple_playlists    902 non-null    int64  
 10  in_apple_charts       902 non-null    int64  
 11  in_deezer_playlists   902 non-null    float64
 12  in_deezer_charts      902 non-null    int64  
 13  in_shazam_charts      902 non-null    float64
 14  bpm                   902 non-null    int64  
 15  key                   816 no

# train/test split

In [ ]:
from sklearn.model_selection import train_test_split
np.random.seed(42)

In [191]:
cat_cols = df.select_dtypes(include='object')
cat_cols.describe()

,track_name,artist(s)_name,key,mode,cover_url
count,902,902,816,902,902
unique,892,476,11,2,501
top,Let It Snow! Let It Snow! Let It Snow!,The Weeknd,C#,Major,Not Found
freq,2,31,115,521,219


Убираем из данных колонки:
 - `track_name`
 - `artist(s)_name`
 - `cover_url`

так как количество уникальных категорий соизмеримо с количеством наблюдений (по этим фичам недостаточная выборка)

Колонки `key`, `mode` будем кодировать с помощью One-Hot-Encoding т. к. на них нет отношения порядка

In [281]:
one_hot_cols = ['key', 'mode']

In [ ]:
X, y = df.drop(
    [
        'artist(s)_name',
        'track_name',
        'cover_url',
        'streams',  # целевое значение
    ], axis=1), df['streams']

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(
    X, y, test_size=0.3, random_state=42)  # Фиксируем random_state

In [202]:
train_X.shape, test_X.shape, train_y.shape, test_y.shape

((631, 21), (271, 21), (631,), (271,))

# Dummy predictions

In [271]:
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyRegressor

from sklearn.pipeline import Pipeline

In [278]:
dummy = DummyRegressor(strategy='mean')
dummy.fit(train_X, train_y)

DummyRegressor()

In [279]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from typing import Dict


def calculate_metrics(predictor) -> Dict[str, float]:
    return {
        'MSE': mean_squared_error(test_y, predictor.predict(test_X)),
        'MAPE': mean_absolute_percentage_error(test_y, predictor.predict(test_X)),
        'R2': r2_score(test_y, predictor.predict(test_X))
    }

In [280]:
for name, metric in calculate_metrics(dummy).items():
    print(f'{name}: {metric}')

MSE: 2.5807544583401706e+17
MAPE: 2.3686679178309307
R2: -0.0017399091513614628


Получили бейзлайн (стоит учесть, что `R2` для идеального константного регрессора равен 0)

# baseline model

In [282]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

Будем использовать `RobustScaler` для препроцессинга, чтобы нивелировать выбросы в обучающей выборке

In [283]:
baseline: Dict[str, Pipeline] = dict(
    linear=Pipeline([
        ('preprocessor', ColumnTransformer([
            ('scaler', RobustScaler(), list(
                set(train_X.columns).difference(one_hot_cols))),
            ('one-hot', OneHotEncoder(handle_unknown='ignore'), one_hot_cols)
        ])),
        ('regressor', LinearRegression())
    ]),
    tree=Pipeline([
        ('preprocessor', ColumnTransformer([
            ('scaler', RobustScaler(), list(
                set(train_X.columns).difference(one_hot_cols))),
            ('one-hot', OneHotEncoder(handle_unknown='ignore'), one_hot_cols)
        ])),
        ('regressor', DecisionTreeRegressor())
    ]),
    knn=Pipeline([
        ('preprocessor', ColumnTransformer([
            ('scaler', RobustScaler(), list(
                set(train_X.columns).difference(one_hot_cols))),
            ('one-hot', OneHotEncoder(handle_unknown='ignore'), one_hot_cols)
        ])),
        ('regressor', KNeighborsRegressor())
    ]),
    random_forest=Pipeline([
        ('preprocessor', ColumnTransformer([
            ('scaler', RobustScaler(), list(
                set(train_X.columns).difference(one_hot_cols))),
            ('one-hot', OneHotEncoder(handle_unknown='ignore'), one_hot_cols)
        ])),
        ('regressor', RandomForestRegressor())
    ])
)

In [284]:
for name, estimator in baseline.items():
    estimator.fit(train_X, train_y)
    print(f'________{name}________')
    for metric, value in calculate_metrics(estimator).items():
        print(f'{metric}: {value}')

________linear________
MSE: 8.646130586213565e+16
MAPE: 1.0119777396532401
R2: 0.6643937186680333
________tree________
MSE: 9.543678378170045e+16
MAPE: 0.5919382470522244
R2: 0.6295547032526824
________knn________
MSE: 1.022643951102742e+17
MAPE: 0.9108973923926973
R2: 0.6030527990133882
________random_forest________
MSE: 5.482432051730795e+16
MAPE: 0.508088002471023
R2: 0.7871951371552984


## Вывод

По всем метрикам побеждает алгоритм случайного леса, выбираем его. Однако стоит помнить, что применение этого алгоритма необходимо делать в отрезке между минимум и максимумом обучающей выборки (так как дерево принятия решения не имеет свойства экстраполяции).